In [1]:
# Data manipulation
import os
import pandas as pd
import csv
import json
import jsonlines
import jsonlines as jl
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
from dotenv import load_dotenv
from typing import List
import re

# Machine Learning
import torch
import asyncio
import aiohttp
import torch.nn as nn
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Transformers and Langchain
from pydantic import ValidationError, BaseModel, Field

# API and utility
from huggingface_hub import login
from together import Together
import time
from tqdm import tqdm
import accelerate

load_dotenv('/Users/guida/llm_argument_tasks/.env')

api_key = os.environ.get('TOGETHER_API_KEY')
client = Together(api_key=api_key)

In [2]:
class ArgumentClassification(BaseModel):
    id: str = Field(description="The exact ID of the comment being analyzed")
    label: int = Field(description="The label associated with the argument (0 or 1)")

In [3]:
# Dictionary for label-to-argument mappings for each topic
topic_label_to_argument = {
    "abortion": {
        "p-right": "Abortion is a woman’s right.",
        "p-rape": "Rape victims need it to be legal.",
        "p-not_human": "A fetus is not a human yet, so it's okay to abort.",
        "p-mother_danger": "Abortion should be allowed when a mother's life is in danger.",
        "p-baby_ill_treatment": "Unwanted babies are ill-treated by parents and/or not always adopted.",
        "p-birth_ctrl": "Birth control fails at times and abortion is one way to deal with it.",
        "p-not_murder": "Abortion is not murder.",
        "p-sick_mom": "Mother is not healthy/financially solvent.",
        "p-other": "Others",
        "c-adopt": "Put baby up for adoption.",
        "c-kill": "Abortion kills a life.",
        "c-baby_right": "An unborn baby is a human and has the right to live.",
        "c-sex": "Be willing to have the baby if you have sex.",
        "c-bad_4_mom": "Abortion is harmful for women.",
        "c-other": "Others"
    },
    "gayRights": {
        "p-normal": "Gay marriage is like any other marriage.",
        "p-right_denied": "Gay people should have the same rights as straight people.",
        "p-no_threat_for_child": "Gay parents can adopt and ensure a happy life for a baby.",
        "p-born": "People are born gay.",
        "p-religion": "Religion should not be used against gay rights.",
        "p-Other": "Others",
        "c-religion": "Religion does not permit gay marriages.",
        "c-abnormal": "Gay marriages are not normal/against nature.",
        "c-threat_to_child": "Gay parents cannot raise kids properly.",
        "c-gay_problems": "Gay people have problems and create social issues.",
        "c-Other": "Others"
    },
    "obama": {
        "p-economy": "Fixed the economy.",
        "p-War": "Ending the wars.",
        "p-republicans": "Better than the republican candidates.",
        "p-decision_policies": "Makes good decisions/policies.",
        "p-quality": "Has qualities of a good leader.",
        "p-health": "Ensured better healthcare.",
        "p-foreign_policies": "Executed effective foreign policies.",
        "p-job": "Created more jobs.",
        "p-Other": "Others",
        "c-economy": "Destroyed our economy.",
        "c-War": "Wars are still on.",
        "c-job": "Unemployment rate is high.",
        "c-health": "Healthcare bill is a failure.",
        "c-decision_policies": "Poor decision-maker.",
        "c-republicans": "We have better republicans than Obama.",
        "c-quality": "Not eligible as a leader.",
        "c-foreign_policies": "Ineffective foreign policies.",
        "c-Other": "Others"
    },
    "marijuana": {
        "p-not_addictive": "Not addictive.",
        "p-medicine": "Used as a medicine for its positive effects.",
        "p-legal": "Legalized marijuana can be controlled and regulated by the government.",
        "p-right": "Prohibition violates human rights.",
        "p-no_damage": "Does not cause any damage to our bodies.",
        "p-Other": "Others",
        "c-health": "Damages our bodies.",
        "c-mind": "Responsible for brain damage.",
        "c-illegal": "If legalized, people will use marijuana and other drugs more.",
        "c-crime": "Causes crime.",
        "c-addiction": "Highly addictive.",
        "c-Other": "Others"
    }
}

In [4]:
def classify_text(id: str, comment_text: str, topic: str, argument_text: str) -> dict:
    #print(f"Analyzing argument: {argument_text}")

    extract = client.chat.completions.create(
        messages=[
            {"role": "system", "content": f"""
            Analyze the given comment in relation to a specific argument about {topic}. You need to:
            Identify if the comment makes use of the given argument. If it does, assign the label 1. If it does not, assign the label 0.
            Do NOT use any other label.
            Do NOT include the comment or the argument in the response.
            
            The argument to analyze is: {argument_text}
            
            Provide your response in the following JSON format:
            
            {{
                "id": "{id}",
                "label": "the label for the use of the argument in the comment"
            }}
            
            Analyze the following comment in relation to the given argument:
            """},
            {"role": "user", "content": comment_text},
        ],
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        temperature=0,
        top_k=1,
        top_p=1,
        response_format={
            "type": "json_object",
            "schema": ArgumentClassification.model_json_schema(),
        }
    )
    
    return json.loads(extract.choices[0].message.content)

In [9]:
def process_dataframe_comments(df: pd.DataFrame, topic: str, output_file: str) -> List[dict]:
    label_to_argument = topic_label_to_argument.get(topic, {}) 
    with jsonlines.open(f'yru_{topic}_identification_with_negatives.jsonl', mode='a') as writer:
        for idx, row in tqdm(df.iterrows(), desc="Processing comments", unit="comment", total=len(df)):
            comment_id = row['uid'] 
            comment_text = row['text']  
            comment_label = row['label']  

            argument_text = label_to_argument.get(comment_label)
            try:
                classification = classify_text(
                    id=comment_id, 
                    comment_text=comment_text,  
                    topic=topic,
                    argument_text=argument_text
                )

                writer.write(classification)

            except json.JSONDecodeError as e:
                print(f"JSONDecodeError for comment: {comment_text[:50]}... - Error: {e}")
                error_entry = {"id": comment_id, "label": 0}
                writer.write(error_entry)
                continue

            except Exception as e:
                print(f"An unexpected error occurred for comment: {comment_text[:50]}... - Error: {e}")
                error_entry = {"id": comment_id, "label": 0}
                writer.write(error_entry)
                continue

In [33]:
ab = pd.read_csv('../../clean_data/yru_abortion_with_negatives_main.csv')
topic = 'abortion'
output_filename = f'yru_{topic}_identification_negatives.jsonl'

results = process_dataframe_comments(ab, topic, output_filename)

Processing comments:   0%|          | 0/6675 [00:00<?, ?comment/s]

Processing comments: 100%|██████████| 6675/6675 [2:31:28<00:00,  1.36s/comment]  


In [37]:
ma = pd.read_csv('../../clean_data/yru_marijuana_with_negatives_main.csv')
topic = 'marijuana'
output_filename = f'yru_{topic}_identification_negatives.jsonl'

results = process_dataframe_comments(ma, topic, output_filename)

Processing comments:  34%|███▎      | 1680/5004 [37:12<1:03:22,  1.14s/comment]

An unexpected error occurred for comment: weeed for life man without potsmokers there wolud ... - Error: Error code: 400 - {"message": "Bad Request", "type_": "invalid_request_error"}


Processing comments:  49%|████▉     | 2464/5004 [54:38<47:47,  1.13s/comment]  

An unexpected error occurred for comment: Sure, if negative health effects are funny. It  ha... - Error: Error code: 400 - {"message": "Bad Request", "type_": "invalid_request_error"}


Processing comments: 100%|██████████| 5004/5004 [1:51:21<00:00,  1.34s/comment]  


In [38]:
oba = pd.read_csv('../../clean_data/yru_obama_with_negatives_main.csv')
topic = 'obama'
output_filename = f'yru_{topic}_identification_negatives.jsonl'

results = process_dataframe_comments(oba, topic, output_filename)

Processing comments:   0%|          | 0/7902 [00:00<?, ?comment/s]

Processing comments:  85%|████████▌ | 6737/7902 [2:29:52<26:22,  1.36s/comment]  

An unexpected error occurred for comment: I don't think we can attribute all of this growth ... - Error: Error code: 400 - {"message": "Bad Request", "type_": "invalid_request_error"}


Processing comments: 100%|██████████| 7902/7902 [2:55:55<00:00,  1.34s/comment]


In [10]:
gm = pd.read_csv('../../clean_data/yru_gayrights_with_negatives_main.csv')
topic = 'gayRights'
output_filename = f'yru_{topic}_identification_negatives.jsonl'

process_dataframe_comments(gm, topic, output_filename)

Processing comments:   0%|          | 0/1491 [00:00<?, ?comment/s]

Processing comments: 100%|██████████| 1491/1491 [32:57<00:00,  1.33s/comment]
